In [1]:
import dgl
import torch
import numpy as np
import torch.nn.functional as F
# 其中包括激活函数, 损失函数, 池化函数 ,通过 F.xxx() 的形式，可以方便地调用 torch.nn.functional 模块中的各种函数
import numpy
import argparse
import time
from pygod.utils import load_data
from dataset_process.local_graph_spectral import get_neighbor, neighbor_pad_or_truncate, neighbor_pad_or_truncate_list, gen_joint_structural_outlier
from sklearn.metrics import f1_score, accuracy_score, recall_score, roc_auc_score, precision_score, confusion_matrix
from sklearn.model_selection import train_test_split
from pygod.generator import gen_contextual_outlier, gen_structural_outlier

from model.BWGNN import *
from sklearn.metrics import f1_score, accuracy_score, recall_score, roc_auc_score, precision_score, confusion_matrix
from torch_geometric.utils import add_self_loops

In [2]:
# 1. 初始化超参数
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('--dataset', type=str, default="books") #"weibo"，"inj_cora"，"enron","disney","books","reddit"
#是否 normalize features
parser.add_argument('--normalize_feat', type=bool, default=True)
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=14)
parser.add_argument('--contextual_k', type=int, default=5)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=14)
parser.add_argument('--structural_m', type=int, default=5)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument("--order", type=int, default=2, help="Order C in Beta Wavelet")

#contextual_n =183  #weibo: 434 # books 14 # cora: 70；dinesy: 3; reddit: 183; Enron: 3
#contextual_k =30  #weibo: 10 # books 5 # Cora: 10；dinesy: 5; reddit: 30; Enron: 25
#structural_n =183  #weibo: 434 # books 14 # Cora: 70；dinesy: 3; reddit: 183; Enron: 3,
#structural_m =30  #weibo: 10 # books 5 # Cora: 10；dinesy: 5; reddit: 30; Enron: 25,

#reddit: 64+32; disney:28+32; weibo: 400+32, enron: 18+32; books:21+32; inj_cora:1433+32  ;
parser.add_argument("--hid_dim_c", type=int, default=64, help="Hidden layer dimension") 
parser.add_argument("--hid_dim_s", type=int, default=32, help="Hidden layer dimension") 
parser.add_argument("--hid_dim_j", type=int, default=32, help="Hidden layer dimension") 

parser.add_argument("--train_ratio", type=float, default=0.4, help="Training ratio")
parser.add_argument("--epoch", type=int, default=500, help="The max number of epochs") # For yelp，epoch =200; For amazon/tfinance/tsocial，epoch =100
parser.add_argument("--run", type=int, default=1, help="Running times")
from torch_geometric.utils import to_scipy_sparse_matrix
from scipy.sparse.linalg import eigs

args = parser.parse_args(args = [])

#2. 读取数据集
dataset_str = args.dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = load_data(dataset_str)

# 计算最大特征值
edge_index = data.edge_index
num_nodes = data.num_nodes  # 如果需要提供节点数量
edge_index_tensor = to_scipy_sparse_matrix(edge_index, num_nodes=num_nodes)
#max_eigenvalue = eigs(edge_index_tensor, k=1, which='LR')[0][0].real

In [3]:
#3. 节点特征归一化
node_features = data.x 
if args.normalize_feat:
    #特征归一化
    node_features_min = node_features.min()
    node_features_max = node_features.max()
    node_features = (node_features - node_features_min)/node_features_max
    data.x = node_features

In [4]:
#4.计算local graph features 并归一化
# a.产生结构、属性和joint anomaly
calculate_contextual=args.calculate_contextual
calculate_structural=args.calculate_structural
yc = []
ys = []
yj = []
    
if calculate_contextual:
        
    if dataset_str == "inj_cora":
        yc = data.y >> 0 & 1 # contextual outliers
    else:
        data, yc = gen_contextual_outlier(data=data,n=args.contextual_n,k=args.contextual_k)
            
    yc = yc.cpu().detach()
    print("Yc:",yc.shape,sum(yc))
    
    
if calculate_structural:
        
    if dataset_str == "inj_cora":
        ys = data.y >> 1 & 1 # structural outliers
    else:
        data, ys = gen_structural_outlier(data=data,n=args.structural_n,m=args.structural_m,p=0.2)
            
    data, yj = gen_joint_structural_outlier(data=data,n=args.structural_n,m=args.structural_m)
        
    
if args.use_combine_outlier:
    data.y = torch.logical_or(ys, yc).int()
        
ysj = torch.logical_or(ys, yj).int()
data.y = data.y.bool()    # binary labels (inlier/outlier)

ys = ys.bool()
yj = yj.bool()
#添加自环
data.edge_index, _ = add_self_loops(data.edge_index)

Yc: torch.Size([1418]) tensor(14)


In [5]:
# b. 按键值排序，形成Substructure 特征
dim = 100
neighbor_dict, neighbor_num_list = get_neighbor(data,device)
# 计算节点与所有邻居的度相似度并将其作为特征
# 1. 按键排序，形成local graph spectral 特征
sorted_tensors = []
for i in range(data.x.shape[0]):
    if i in neighbor_dict.keys():
        degree_sims = []
        for j in neighbor_dict[i]:
            common_set = set(neighbor_dict[i]) & set(neighbor_dict[j])
            count = len(common_set)
            degree_sim = count / math.sqrt(len(neighbor_dict[i])*len(neighbor_dict[j]))
            degree_sims.append(degree_sim)
        sorted_tensors.append(degree_sims)
    else:
        vector = torch.zeros(dim)
        sorted_tensors.append(vector)
    
neighbor_degree_simiarity = np.array(neighbor_pad_or_truncate_list(sorted_tensors, dim))

In [6]:
# For reddit
# node_features:            torch.Size([10984, 64])
# neighbor_spectral_tensor：torch.Size([10984, 32])
#e. 归一化
if args.normalize_feat:
    #特征归一化
    neighbor_degree_simiarity =  torch.tensor(neighbor_degree_simiarity).float()
    neighbor_degree_simiarity_min = neighbor_degree_simiarity.real.min()
    neighbor_degree_simiarity_max = neighbor_degree_simiarity.real.max()
    neighbor_degree_simiarity_features = (neighbor_degree_simiarity.real - neighbor_degree_simiarity_min)/neighbor_degree_simiarity_max

In [7]:
graph = dgl.graph((edge_index[0], edge_index[1]), num_nodes=num_nodes)
graph.ndata['feat'] = data.x

In [8]:
def train(model_c, model_s, model_j, data, neighbor_degree_simiarity_features, ys, yj, args):
    # contextual_outlier
    features_c = data.x
    labels_c = data.y
     # structural_outlier
    features_s = neighbor_degree_simiarity_features
    labels_s = ys
     # jiont_outlier
    features_j = neighbor_degree_simiarity_features
    labels_j = yj
    
    index = list(range(len(labels_c)))

    idx_train_c, idx_rest_c, y_train_c, y_rest_c = train_test_split(index, labels_c[index], stratify=labels_c[index],
                                                            train_size=args.train_ratio,
                                                            random_state=2, shuffle=True)
    idx_valid_c, idx_test_c, y_valid_c, y_test_c = train_test_split(idx_rest_c, y_rest_c, stratify=y_rest_c,
                                                            test_size=0.67,
                                                            random_state=2, shuffle=True)
    idx_train_s, idx_rest_s, y_train_s, y_rest_s = train_test_split(index, labels_s[index], stratify=labels_s[index],
                                                            train_size=args.train_ratio,
                                                            random_state=2, shuffle=True)
    idx_valid_s, idx_test_s, y_valid_s, y_tes_s = train_test_split(idx_rest_s, y_rest_s, stratify=y_rest_s,
                                                            test_size=0.67,
                                                            random_state=2, shuffle=True)
    idx_train_j, idx_rest_j, y_train_j, y_rest_j = train_test_split(index, labels_j[index], stratify=labels_j[index],
                                                            train_size=args.train_ratio,
                                                            random_state=2, shuffle=True)
    idx_valid_j, idx_test_j, y_valid_j, y_tes_j = train_test_split(idx_rest_j, y_rest_j, stratify=y_rest_j,
                                                            test_size=0.67,
                                                            random_state=2, shuffle=True)
    train_mask_c = torch.zeros([len(labels_c)]).bool()
    val_mask_c = torch.zeros([len(labels_c)]).bool()
    test_mask_c = torch.zeros([len(labels_c)]).bool()
    train_mask_s = torch.zeros([len(labels_s)]).bool()
    val_mask_s = torch.zeros([len(labels_s)]).bool()
    test_mask_s = torch.zeros([len(labels_s)]).bool()
    train_mask_j = torch.zeros([len(labels_j)]).bool()
    val_mask_j = torch.zeros([len(labels_j)]).bool()
    test_mask_j = torch.zeros([len(labels_j)]).bool()
    print(" y_train:", y_train_c.shape, sum(y_train_c))
    
    train_mask_c[idx_train_c] = 1
    val_mask_c[idx_valid_c] = 1
    test_mask_c[idx_test_c] = 1
    train_mask_s[idx_train_s] = 1
    val_mask_s[idx_valid_s] = 1
    test_mask_s[idx_test_s] = 1
    train_mask_j[idx_train_j] = 1
    val_mask_j[idx_valid_j] = 1
    test_mask_j[idx_test_j] = 1
    print('train/dev/test samples c: ', train_mask_c.sum().item(), val_mask_c.sum().item(), test_mask_c.sum().item())
    print('train/dev/test samples  s: ', train_mask_s.sum().item(), val_mask_s.sum().item(), test_mask_s.sum().item())
    print('train/dev/test samples  j: ', train_mask_j.sum().item(), val_mask_j.sum().item(), test_mask_j.sum().item())
    
    
    optimizer_c = torch.optim.Adam(model_c.parameters(), lr=0.01)
    optimizer_s = torch.optim.Adam(model_s.parameters(), lr=0.01)
    optimizer_j = torch.optim.Adam(model_j.parameters(), lr=0.01)
    
    best_f1_c, final_tf1_c, final_trec_c, final_tpre_c, final_tmf1_c, final_tauc_c = 0., 0., 0., 0., 0., 0.
    best_f1_s, final_tf1_s, final_trec_s, final_tpre_s, final_tmf1_s, final_tauc_s = 0., 0., 0., 0., 0., 0.
    best_f1_j, final_tf1_j, final_trec_j, final_tpre_j, final_tmf1_j, final_tauc_j = 0., 0., 0., 0., 0., 0.


    #对bool型的不允许“-”操作1-labels[train_mask] 改为：~labels[train_mask]
    weight_c = (~labels_c[train_mask_c]).sum().item() / labels_c[train_mask_c].sum().item()
    weight_s = (~labels_s[train_mask_s]).sum().item() / labels_s[train_mask_s].sum().item()
    weight_j = (~labels_j[train_mask_j]).sum().item() / labels_j[train_mask_j].sum().item()
    print('cross entropy weight: ', weight_c, weight_s, weight_j)
    time_start = time.time()
    for e in range(args.epoch):
        # 训练
        model_c.train()
        model_s.train()
        model_j.train()
        
        # 调用模型中的forward函数
        logits_c = model_c(features_c)
        logits_s = model_s(features_s)
        logits_j = model_j(features_j)
        
        #labels[train_mask] 需要long型
        loss_c = F.cross_entropy(logits_c[train_mask_c], labels_c[train_mask_c].to(torch.long), weight = torch.tensor([1., weight_c]))
        loss_s = F.cross_entropy(logits_s[train_mask_s], labels_s[train_mask_s].to(torch.long), weight = torch.tensor([1., weight_s]))
        loss_j = F.cross_entropy(logits_j[train_mask_j], labels_j[train_mask_j].to(torch.long), weight = torch.tensor([1., weight_j]))
        
        optimizer_c.zero_grad()
        optimizer_s.zero_grad()
        optimizer_j.zero_grad()
        
        loss_c.backward()
        loss_s.backward()
        loss_j.backward()
        
        optimizer_c.step()
        optimizer_s.step()
        optimizer_j.step()
        
        #验证
        model_c.eval()
        model_s.eval()
        model_j.eval()
        probs_c = logits_c.softmax(1)
        probs_s = logits_s.softmax(1)
        probs_j = logits_j.softmax(1)
        
        f1_c, thres_c = get_best_f1(labels_c[val_mask_c], probs_c[val_mask_c])
        f1_s, thres_s = get_best_f1(labels_s[val_mask_s], probs_s[val_mask_s])
        f1_j, thres_j = get_best_f1(labels_j[val_mask_j], probs_j[val_mask_j])
        
        preds_c = numpy.zeros_like(labels_c)
        preds_c[probs_c[:, 1] > thres_c] = 1
        preds_s = numpy.zeros_like(labels_s)
        preds_s[probs_s[:, 1] > thres_s] = 1
        preds_j = numpy.zeros_like(labels_j)
        preds_j[probs_j[:, 1] > thres_j] = 1

        trec_c = recall_score(labels_c[test_mask_c], preds_c[test_mask_c])
        tpre_c = precision_score(labels_c[test_mask_c], preds_c[test_mask_c])
        tmf1_c = f1_score(labels_c[test_mask_c], preds_c[test_mask_c], average='macro')
        tauc_c = roc_auc_score(labels_c[test_mask_c], probs_c[test_mask_c][:, 1].detach().numpy())
        
        trec_s = recall_score(labels_s[test_mask_s], preds_s[test_mask_s])
        tpre_s = precision_score(labels_s[test_mask_s], preds_s[test_mask_s])
        tmf1_s = f1_score(labels_s[test_mask_s], preds_s[test_mask_s], average='macro')
        tauc_s = roc_auc_score(labels_s[test_mask_s], probs_s[test_mask_s][:, 1].detach().numpy())

        trec_j = recall_score(labels_j[test_mask_j], preds_j[test_mask_j])
        tpre_j = precision_score(labels_j[test_mask_j], preds_j[test_mask_j])
        tmf1_j = f1_score(labels_j[test_mask_j], preds_j[test_mask_j], average='macro')
        tauc_j = roc_auc_score(labels_j[test_mask_j], probs_j[test_mask_j][:, 1].detach().numpy())

        if best_f1_c < f1_c:
            best_f1_c = f1_c
            final_trec_c = trec_c
            final_tpre_c = tpre_c
            final_tmf1_c = tmf1_c
            final_tauc_c = tauc_c
        print('C: Epoch {}, loss: {:.4f}, val mf1: {:.4f}, (best {:.4f})'.format(e, loss_c, f1_c, best_f1_c))

        if best_f1_s < f1_s:
            best_f1_s = f1_s
            final_trec_s = trec_s
            final_tpre_s = tpre_s
            final_tmf1_s = tmf1_s
            final_tauc_s = tauc_s
        print('s: Epoch {}, loss: {:.4f}, val mf1: {:.4f}, (best {:.4f})'.format(e, loss_s, f1_s, best_f1_s))

        if best_f1_j < f1_j:
            best_f1_j = f1_j
            final_trec_j = trec_j
            final_tpre_j = tpre_j
            final_tmf1_j = tmf1_j
            final_tauc_j = tauc_j
        print('j: Epoch {}, loss: {:.4f}, val mf1: {:.4f}, (best {:.4f})'.format(e, loss_j, f1_j, best_f1_j))

    time_end = time.time()
    print('time cost: ', time_end - time_start, 's')
    print('Test_c: REC {:.2f} PRE {:.2f} MF1 {:.2f} AUC {:.2f}'.format(final_trec_c*100,
                                                                     final_tpre_c*100, final_tmf1_c*100, final_tauc_c*100))
    print('Test_s: REC {:.2f} PRE {:.2f} MF1 {:.2f} AUC {:.2f}'.format(final_trec_s*100,
                                                                     final_tpre_s*100, final_tmf1_s*100, final_tauc_s*100))
    print('Test_j: REC {:.2f} PRE {:.2f} MF1 {:.2f} AUC {:.2f}'.format(final_trec_j*100,
                                                                     final_tpre_j*100, final_tmf1_j*100, final_tauc_j*100))

    return final_tmf1_c, final_tauc_c, final_tmf1_s, final_tauc_s, final_tmf1_j, final_tauc_j


# threshold adjusting for best macro f1
def get_best_f1(labels, probs):
    best_f1, best_thre = 0, 0
    for thres in np.linspace(0.05, 0.95, 19):
        #构建一个与labels同维度的数组,并初始化所有变量为零
        preds = np.zeros_like(labels)
        preds[probs[:,1] > thres] = 1
        #average='binary'：计算二分类问题中的 F1 分数（默认值）。
        #average='micro'：对所有类别的真实和预测样本进行汇总，然后计算 F1 分数。
        #average='macro'：计算每个类别的 F1 分数，然后取平均值。
        #average=None：返回每个类别的 F1 分数。
        # F1_score 详细原理间“备份”
        mf1 = f1_score(labels, preds, average='macro')
        if mf1 > best_f1:
            best_f1 = mf1
            best_thre = thres
    return best_f1, best_thre


In [9]:
h_feats_c = args.hid_dim_c
in_feats_c = data.x.shape[1]
h_feats_s = args.hid_dim_s
in_feats_s = neighbor_degree_simiarity_features.shape[1]
h_feats_j = args.hid_dim_j
in_feats_j = neighbor_degree_simiarity_features.shape[1]
num_classes = 2

if args.run == 0:
    model_c = BWGNN_C(in_feats_c, h_feats_c, num_classes, graph,  d = args.order)
    model_s = BWGNN_S(in_feats_s, h_feats_s, num_classes, graph,  d = args.order)
    model_j = BWGNN_J(in_feats_j, h_feats_j, num_classes, graph,  d = args.order)
    
    train(model_c, model_s, model_j, data, neighbor_degree_simiarity_features, ys, yj, args)

else:
    final_mf1s_c, final_aucs_c,final_mf1s_s, final_aucs_s, final_mf1s_j, final_aucs_j= [], [], [], [], [], []
    for tt in range(args.run):

        #in_feats 特征点维度；h_feats：隐层维度；num_classes：节点分类数（nomal，anomaly）
        model_c = BWGNN_C(in_feats_c, h_feats_c, num_classes, graph, d = args.order)
        model_s = BWGNN_S(in_feats_s, h_feats_s, num_classes, graph, d = args.order)
        model_j = BWGNN_J(in_feats_j, h_feats_j, num_classes, graph, d = args.order)
        mf1_c, auc_c, mf1_s, auc_s, mf1_j, auc_j = train(model_c, model_s, model_j, data, neighbor_degree_simiarity_features, ys, yj, args)
        
        final_mf1s_c.append(mf1_c)
        final_aucs_c.append(auc_c)
        final_mf1s_s.append(mf1_s)
        final_aucs_s.append(auc_s)
        final_mf1s_j.append(mf1_j)
        final_aucs_j.append(auc_j)
    final_mf1s_c = np.array(final_mf1s_c)
    final_aucs_c = np.array(final_aucs_c)
    # np.std :计算全局标准差
    print('MF1-mean_c: {:.2f}, MF1-std: {:.2f}, AUC-mean: {:.2f}, AUC-std: {:.2f}'.format(100 * np.mean(final_mf1s_c),
    100 * np.std(final_mf1s_c),
                                                               100 * np.mean(final_aucs_c), 100 * np.std(final_aucs_c)))

    final_mf1s_s = np.array(final_mf1s_s)
    final_aucs_s = np.array(final_aucs_s)
    # np.std :计算全局标准差
    print('MF1-mean_s: {:.2f}, MF1-std: {:.2f}, AUC-mean: {:.2f}, AUC-std: {:.2f}'.format(100 * np.mean(final_mf1s_s),
                                                                                            100 * np.std(final_mf1s_s),
                                                               100 * np.mean(final_aucs_s), 100 * np.std(final_aucs_s)))

    final_mf1s_j = np.array(final_mf1s_j)
    final_aucs_j = np.array(final_aucs_j)
    # np.std :计算全局标准差
    print('MF1-mean_j: {:.2f}, MF1-std: {:.2f}, AUC-mean: {:.2f}, AUC-std: {:.2f}'.format(100 * np.mean(final_mf1s_j),
                                                                                            100 * np.std(final_mf1s_j),
                                                               100 * np.mean(final_aucs_j), 100 * np.std(final_aucs_j)))

 y_train: torch.Size([567]) tensor(11)
train/dev/test samples c:  567 280 571
train/dev/test samples  s:  567 280 571
train/dev/test samples  j:  567 280 571
cross entropy weight:  50.54545454545455 19.25 93.5
C: Epoch 0, loss: 0.6889, val mf1: 0.4946, (best 0.4946)
s: Epoch 0, loss: 0.7056, val mf1: 0.4872, (best 0.4872)
j: Epoch 0, loss: 0.6945, val mf1: 0.4973, (best 0.4973)


C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_clas

C: Epoch 1, loss: 0.6628, val mf1: 0.4946, (best 0.4946)
s: Epoch 1, loss: 0.6897, val mf1: 0.4918, (best 0.4918)
j: Epoch 1, loss: 0.6921, val mf1: 0.4973, (best 0.4973)
C: Epoch 2, loss: 0.6503, val mf1: 0.4946, (best 0.4946)
s: Epoch 2, loss: 0.6824, val mf1: 0.4872, (best 0.4918)
j: Epoch 2, loss: 0.6889, val mf1: 0.4973, (best 0.4973)


C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_clas

C: Epoch 3, loss: 0.6345, val mf1: 0.4946, (best 0.4946)
s: Epoch 3, loss: 0.6758, val mf1: 0.4872, (best 0.4918)
j: Epoch 3, loss: 0.6853, val mf1: 0.4973, (best 0.4973)
C: Epoch 4, loss: 0.6232, val mf1: 0.4946, (best 0.4946)
s: Epoch 4, loss: 0.6609, val mf1: 0.5530, (best 0.5530)
j: Epoch 4, loss: 0.6788, val mf1: 0.4973, (best 0.4973)


...

C: Epoch 449, loss: 0.5086, val mf1: 0.4946, (best 0.5453)
s: Epoch 449, loss: 0.0073, val mf1: 0.8535, (best 0.8866)
j: Epoch 449, loss: 0.0051, val mf1: 0.4955, (best 0.5369)
C: Epoch 450, loss: 0.5071, val mf1: 0.5300, (best 0.5453)
s: Epoch 450, loss: 0.0089, val mf1: 0.8535, (best 0.8866)
j: Epoch 450, loss: 0.0051, val mf1: 0.4955, (best 0.5369)
C: Epoch 451, loss: 0.5052, val mf1: 0.5300, (best 0.5453)
s: Epoch 451, loss: 0.0087, val mf1: 0.8535, (best 0.8866)
j: Epoch 451, loss: 0.0051, val mf1: 0.4955, (best 0.5369)
C: Epoch 452, loss: 0.5056, val mf1: 0.5300, (best 0.5453)
s: Epoch 452, loss: 0.0097, val mf1: 0.8535, (best 0.8866)
j: Epoch 452, loss: 0.0051, val mf1: 0.4955, (best 0.5369)
C: Epoch 453, loss: 0.5051, val mf1: 0.5300, (best 0.5453)
s: Epoch 453, loss: 0.0110, val mf1: 0.8535, (best 0.8866)
j: Epoch 453, loss: 0.0051, val mf1: 0.4955, (best 0.5369)
C: Epoch 454, loss: 0.5039, val mf1: 0.5300, (best 0.5453)
s: Epoch 454, loss: 0.0116, val mf1: 0.8387, (best 0.886

C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


C: Epoch 463, loss: 0.5104, val mf1: 0.4946, (best 0.5453)
s: Epoch 463, loss: 0.0104, val mf1: 0.8535, (best 0.8866)
j: Epoch 463, loss: 0.0051, val mf1: 0.4955, (best 0.5369)
C: Epoch 464, loss: 0.5079, val mf1: 0.5269, (best 0.5453)
s: Epoch 464, loss: 0.0113, val mf1: 0.8535, (best 0.8866)
j: Epoch 464, loss: 0.0051, val mf1: 0.4955, (best 0.5369)
C: Epoch 465, loss: 0.5056, val mf1: 0.5300, (best 0.5453)
s: Epoch 465, loss: 0.0121, val mf1: 0.8413, (best 0.8866)
j: Epoch 465, loss: 0.0051, val mf1: 0.4955, (best 0.5369)
C: Epoch 466, loss: 0.5054, val mf1: 0.5300, (best 0.5453)
s: Epoch 466, loss: 0.0124, val mf1: 0.8413, (best 0.8866)
j: Epoch 466, loss: 0.0051, val mf1: 0.4955, (best 0.5369)
C: Epoch 467, loss: 0.5061, val mf1: 0.5300, (best 0.5453)
s: Epoch 467, loss: 0.0116, val mf1: 0.8413, (best 0.8866)
j: Epoch 467, loss: 0.0051, val mf1: 0.4955, (best 0.5369)
C: Epoch 468, loss: 0.5051, val mf1: 0.5300, (best 0.5453)
s: Epoch 468, loss: 0.0109, val mf1: 0.8413, (best 0.886

C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


C: Epoch 477, loss: 0.5241, val mf1: 0.4946, (best 0.5453)
s: Epoch 477, loss: 0.0123, val mf1: 0.8694, (best 0.8866)
j: Epoch 477, loss: 0.0051, val mf1: 0.4955, (best 0.5369)
C: Epoch 478, loss: 0.5235, val mf1: 0.5211, (best 0.5453)
s: Epoch 478, loss: 0.0114, val mf1: 0.8535, (best 0.8866)
j: Epoch 478, loss: 0.0051, val mf1: 0.4955, (best 0.5369)
C: Epoch 479, loss: 0.5228, val mf1: 0.5211, (best 0.5453)
s: Epoch 479, loss: 0.0107, val mf1: 0.8535, (best 0.8866)
j: Epoch 479, loss: 0.0051, val mf1: 0.4955, (best 0.5369)
C: Epoch 480, loss: 0.5235, val mf1: 0.5211, (best 0.5453)
s: Epoch 480, loss: 0.0101, val mf1: 0.8535, (best 0.8866)
j: Epoch 480, loss: 0.0051, val mf1: 0.4955, (best 0.5369)
C: Epoch 481, loss: 0.5250, val mf1: 0.5211, (best 0.5453)
s: Epoch 481, loss: 0.0096, val mf1: 0.8535, (best 0.8866)
j: Epoch 481, loss: 0.0051, val mf1: 0.4955, (best 0.5369)
C: Epoch 482, loss: 0.5254, val mf1: 0.5211, (best 0.5453)
s: Epoch 482, loss: 0.0089, val mf1: 0.8535, (best 0.886